## Deliverable 2. Create a Customer Travel Destinations Map.

In [125]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [126]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.count()

City_ID                722
City                   722
Country                717
Lat                    722
Lng                    722
Max Temp               722
Humidity               722
Cloudiness             722
Wind Speed             722
Current Description    722
dtype: int64

In [127]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Enter the desired minimum temperature in Fahrenheit you would like for your trip: "))
max_temp = float(input("Enter the desired maximum temperature in Fahrenheit you would like for your trip: "))

Enter the desired minimum temperature in Fahrenheit you would like for your trip: 60
Enter the desired maximum temperature in Fahrenheit you would like for your trip: 75


In [128]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]>= min_temp) & 
                                      (city_data_df["Max Temp"] <= max_temp)].dropna()
preferred_cities_df.count()

City_ID                154
City                   154
Country                154
Lat                    154
Lng                    154
Max Temp               154
Humidity               154
Cloudiness             154
Wind Speed             154
Current Description    154
dtype: int64

In [129]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [130]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [131]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,61.81,clear sky,-33.6500,115.3333,
4,Taoudenni,ML,72.64,scattered clouds,22.6783,-3.9836,
7,Lompoc,US,60.21,clear sky,34.6391,-120.4579,
10,Sinjār,IQ,67.32,clear sky,36.3209,41.8754,
18,Cidreira,BR,72.39,broken clouds,-30.1811,-50.2056,


In [132]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [133]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,61.81,clear sky,-33.6500,115.3333,Observatory Guest House
4,Taoudenni,ML,72.64,scattered clouds,22.6783,-3.9836,
7,Lompoc,US,60.21,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
10,Sinjār,IQ,67.32,clear sky,36.3209,41.8754,زوماني فون 1
18,Cidreira,BR,72.39,broken clouds,-30.1811,-50.2056,Hotel Castelo


In [134]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Busselton,AU,61.81,clear sky,-33.6500,115.3333,Observatory Guest House
7,Lompoc,US,60.21,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
10,Sinjār,IQ,67.32,clear sky,36.3209,41.8754,زوماني فون 1
18,Cidreira,BR,72.39,broken clouds,-30.1811,-50.2056,Hotel Castelo
19,Ponta do Sol,PT,65.30,few clouds,32.6667,-17.1000,Hotel do Campo


In [135]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [140]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} </dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [141]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))